In [7]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
#from tensorflow.keras import layers
#from tensorflow.keras.models import Model
import datetime, os
from keras.models import Model
from keras.layers import Dense, Input




In [12]:

dict_stati= load_file("C:/Users/Francesco/Desktop/dataset/dataset/dizionario_stati")
plansLoaded=load_file("C:/Users/Francesco/Desktop/dataset/dataset/plans")
#print(type(plansPickles))
#plansList=load_from_pickles("C:/Users/Francesco/Desktop/dataset/dataset/dizionario_stati",plansPickles)
#print(plansPickles)


File loaded
File loaded


In [14]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 
def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(vector) 
    r = np.array(vector_states)                          
    return r


#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani
def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard
def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(vector)
    total=np.array(total)        
    return total        

In [47]:
#all_states_from_plans_list=build_all_vectors(dict_stati,plansLoaded)

## Preparazione dati

In [15]:
#Creazione dataset totale
all1x340=build_all_vectors1x340(dict_stati,plansLoaded)
random.shuffle(all1x340)


In [18]:
print(all1x340.shape)
print(type(all1x340))
print(type(all1x340[6]))
print(all1x340[6].dtype)

(1384373, 340)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
int8


In [17]:
#creAzione di train, test e validation set,--> input shape per rete = 340
#rendere questi set statici, altimenti cambiano sempre ad ogni esecuzione.... es. comprimerli con 7zip o pickle --> DA FARE IN PRIMA POSSIBILE
#random.shuffle(all1x340)
half=int(len(all1x340)//3)
train=all1x340[half:]
vt=all1x340[:half]
h2=int(len(vt)//2)
validation=vt[h2:]
test=vt[:h2]


In [8]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

## Modello

In [24]:
input_size = 340 #costante

#Scelti casulmente al momento.... da rivedere come prima modifica forse
hidden_size = 85
#hidden_size2 = 85
code_size = 17



my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.TensorBoard(log_dir="C:/Users/Francesco/Desktop/owl-main/logsTf",histogram_freq=1)

]

#Cose da fare per migliorare la rete solo dopo aver fatto i primi tentativi con la rete proposta:
# * Provare ad aggiungere Regolarizzazione es. L1,L2 e dropout(solo nella fase di encoding)
# * Provare swish al posto di relu
# * Provare keras.layers.BatchNormalization()
# * Fare Hyperparameter Tuning  (ultima) www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams
#Salvare i risultati con ogni modifica fatta per scriverli nel report

input_layer = Input(shape=(input_size,))
hidden_1 = Dense(hidden_size, activation='relu', kernel_initializer="he_uniform")(input_layer)
#hidden_3 = Dense(hidden_size, activation='relu', kernel_initializer="he_uniform")(hidden_1)

code = Dense(code_size, activation='relu',kernel_initializer="he_uniform")(hidden_1)

hidden_2 =Dense(hidden_size, activation='relu',kernel_initializer="he_uniform")(code)
#hidden_4 =Dense(hidden_size, activation='relu',kernel_initializer="he_uniform")(hidden_2)
output_layer = Dense(input_size, activation='sigmoid')(hidden_2)

autoencoder = Model(input_layer, output_layer)
#variare learning_rate beta_1,beta_2, da fare per ultimo
#batch_size ???
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='binary_crossentropy',metrics=["accuracy","Precision","Recall"])
autoencoder.fit(x=train,y=train, epochs=5000,batch_size=5000, validation_data=(validation,validation), callbacks=my_callbacks,
                      )

 #Gardare i risulati delle metriche su tensorbord                      

Epoch 1/5000
185/185 [==============================] - 5s 22ms/step - loss: 0.1689 - accuracy: 0.0454 - precision: 0.4060 - recall: 0.0883 - val_loss: 0.1403 - val_accuracy: 0.0513 - val_precision: 0.6572 - val_recall: 0.1589
Epoch 2/5000
185/185 [==============================] - 2s 12ms/step - loss: 0.1345 - accuracy: 0.0509 - precision: 0.6692 - recall: 0.1827 - val_loss: 0.1293 - val_accuracy: 0.0467 - val_precision: 0.6723 - val_recall: 0.2124
Epoch 3/5000
185/185 [==============================] - 2s 12ms/step - loss: 0.1241 - accuracy: 0.0469 - precision: 0.6857 - recall: 0.2415 - val_loss: 0.1188 - val_accuracy: 0.0613 - val_precision: 0.7079 - val_recall: 0.2610
Epoch 4/5000
185/185 [==============================] - 2s 12ms/step - loss: 0.1153 - accuracy: 0.0687 - precision: 0.6961 - recall: 0.2937 - val_loss: 0.1116 - val_accuracy: 0.0876 - val_precision: 0.6998 - val_recall: 0.3143
Epoch 5/5000
185/185 [==============================] - 2s 12ms/step - loss: 0.1090 - accura

In [10]:
#%tensorboard --logdir logsTf
%load_ext tensorboard
#Avvio da terminale: spostarsi nella cartella dove è presente la cartella di log, > tensorboard --logdir nomecartella

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Print di test

In [124]:
print(validation[59])
print(test[2000])
print(train[5000])

tf.Tensor(
[0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0], shape=(340,), dtype=int8)
tf.Tensor(
[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [48]:
for i in range(3):
    print(all_states_from_plans_list[i])

tf.Tensor(
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]], shape=(12, 340), dtype=int8)
tf.Tensor(
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]], shape=(11, 340), dtype=int8)
tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]], shape=(15, 340), dtype=int8)


In [4]:
print("********initial state *******")
print(len(plansLoaded[1].initial_state))
print("")
print(plansLoaded[1].initial_state)

print("******** states *******")
print(len(plansLoaded[1].states))
print("")
print(plansLoaded[1].states)
print("******* goals ********")
print(len(plansLoaded[1].goals))
print("")
print(plansLoaded[1].goals)

print("\n\n -------------  dict stati -----------")
print(len(dict_stati))
print("")
print(dict_stati)

********initial state *******
14

['clear crate3', 'clear pallet0', 'at crate0 distributor0', 'available hoist4', 'at truck0 distributor0', 'available hoist3', 'available hoist2', 'on crate3 pallet4', 'at crate3 depot1', 'on crate0 pallet1', 'available hoist0', 'clear crate0', 'clear pallet2', 'at truck2 distributor2']
******** states *******
11

[['at truck2 depot1', 'clear crate0', 'available hoist0', 'on crate0 pallet1', 'clear crate3', 'on crate3 pallet4', 'available hoist4', 'at crate0 distributor0', 'available hoist2', 'available hoist3', 'at crate3 depot1', 'clear pallet0', 'clear pallet2', 'at truck0 distributor0'], ['at truck2 depot1', 'lifting hoist2 crate0', 'clear pallet1', 'available hoist0', 'clear crate3', 'on crate3 pallet4', 'available hoist4', 'available hoist3', 'at crate3 depot1', 'clear pallet0', 'clear pallet2', 'at truck0 distributor0'], ['at truck2 depot1', 'clear pallet1', 'in crate0 truck0', 'available hoist0', 'clear crate3', 'on crate3 pallet4', 'available h

In [64]:
for i in range(5):
    print(all1x340[i])

tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0], shape=(340,), dtype=int8)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 